In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Generate Timestamps from January 2018 to December 2021 with hourly frequency
start_date = '2018-01-01'
end_date = '2021-12-31'
timestamps = pd.date_range(start=start_date, end=end_date, freq='H')
np.random.seed(42)

In [9]:
# Generate CPUTime: Mean = 350000, with fluctuations based on time of day, holidays, and random spikes
def generate_cputime(timestamps):
    # Seasonal pattern with drops during Christmas holidays, exams, and summer months
    holiday_pattern = (
        np.isin(timestamps.month, [12, 6, 9]) |
        (timestamps.month == 7) | (timestamps.month == 8)
    ) * -90000

    # Random spikes
    spikes = np.clip(np.random.lognormal(mean=5, sigma=3, size=len(timestamps)), None, 5e6)

    # Noise
    noise = np.random.normal(loc=0, scale=50000, size=len(timestamps)) 

    #cycle
    cycle = np.sin(np.arange(0, len(timestamps)) * 2 * np.pi / 24) * 50000

    series = 1000000 + holiday_pattern + spikes + cycle + noise

    return np.clip(series, 0, None)

In [10]:
# Generate MemoryUsed: High correlation with CPUTime but non-linear relationship
def generate_memoryused(cputime):
    # Add some noise and non-linear relationship to CPUTime
    noise = np.random.normal(loc=0, scale=20000, size=len(cputime))

    memory_ratio = (np.sin(cputime / 800000) * 50) + 100

    return np.clip(cputime * memory_ratio + noise, 0, None)

In [11]:
# Generate NJobs: Non-linear relationship with CPUTime and MemoryUsed
def generate_njobs(cputime, memoryused):
    # Non-linear relationship with CPUTime and MemoryUsed
    cputime = np.clip(cputime, 1e-8, None)
    cputime_factor = np.clip(np.log(cputime), -10, None) 
    memoryused_factor = np.sqrt(memoryused)*np.tanh(memoryused / 1e9) / 10

    # Random fluctuations with some correlation to CPUTime and MemoryUsed
    njobs_noise = np.random.normal(loc=15, scale=10, size=len(cputime))
    njobs = cputime_factor + memoryused_factor + njobs_noise

    return np.clip(njobs, 1, 500)  # Clip to a minimum of 1 and maximum of 100 jobs


In [12]:
def generate_nprocs(njobs):
    nprocs_random_factor = np.random.normal(loc=6, scale=3, size=len(njobs))

    nprocs = njobs * np.clip(nprocs_random_factor, 1, None)
    return nprocs 

In [13]:
def generate_jobs(timestamps):
    arrived_jobs = np.random.poisson(lam=10, size=len(timestamps))
    completed_jobs = np.random.poisson(lam=8, size=len(timestamps))
    return arrived_jobs, completed_jobs

In [14]:
# Generate the synthetic dataset
cputime = generate_cputime(timestamps)
memoryused = generate_memoryused(cputime)
njobs = generate_njobs(cputime, memoryused)
nprocs = generate_nprocs(njobs)
arrived_jobs, completed_jobs = generate_jobs(timestamps)

In [15]:
# Combine all features into a DataFrame
data = {
    'Timestamp': timestamps,
    'CPUTime': cputime,
    'Memory': memoryused,
    'NJobs': njobs,
    'NProcs': nprocs,
    'ArrivedJobs': arrived_jobs,
    'CompletedJobs': completed_jobs,
}
dataset = pd.DataFrame(data)

dataset["cosHour"] = dataset["Timestamp"].map(lambda timestamp: np.cos(2*np.pi*timestamp.hour/24))
dataset["sinHour"] = dataset["Timestamp"].map(lambda timestamp: np.sin(2*np.pi*timestamp.hour/24))

dataset["cosDay"] = dataset["Timestamp"].map(lambda timestamp: np.cos(2*np.pi*timestamp.timetuple().tm_yday/365))
dataset["sinDay"] = dataset["Timestamp"].map(lambda timestamp: np.sin(2*np.pi*timestamp.timetuple().tm_yday/365))

dataset["cosMonth"] = dataset["Timestamp"].map(lambda timestamp: np.cos(2*np.pi*timestamp.month/12))
dataset["sinMonth"] = dataset["Timestamp"].map(lambda timestamp: np.sin(2*np.pi*timestamp.month/12))


# dataset.to_csv("./datasets/final/final-formed-synthetic.csv", index=False)

In [16]:
dataset.head()

,Timestamp,CPUTime,Memory,NJobs,NProcs,ArrivedJobs,CompletedJobs,cosHour,sinHour,cosDay,sinDay,cosMonth,sinMonth
0,2018-01-01 00:00:00,1.039633e+06,1.540861e+08,228.026135,1682.249177,5,6,1.000000,0.000000,0.999852,0.017213,0.866025,0.5
1,2018-01-01 01:00:00,1.047930e+06,1.554115e+08,232.836734,875.465388,4,10,0.965926,0.258819,0.999852,0.017213,0.866025,0.5
2,2018-01-01 02:00:00,1.034680e+06,1.532273e+08,223.195183,1280.366604,6,8,0.866025,0.500000,0.999852,0.017213,0.866025,0.5
3,2018-01-01 03:00:00,1.030387e+06,1.524702e+08,213.018300,2675.856184,18,6,0.707107,0.707107,0.999852,0.017213,0.866025,0.5
4,2018-01-01 04:00:00,1.029641e+06,1.524012e+08,217.920878,217.920878,15,11,0.500000,0.866025,0.999852,0.017213,0.866025,0.5


In [17]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CPUTime,35041.0,9.734367e+05,1.500311e+05,6.915277e+05,9.138773e+05,9.686288e+05,1.022015e+06,6.115376e+06
Memory,35041.0,1.425511e+08,2.047373e+07,9.547545e+07,1.329464e+08,1.421877e+08,1.511115e+08,9.106040e+08
NJobs,35041.0,1.979696e+02,2.855392e+01,1.007669e+02,1.798426e+02,1.969787e+02,2.144729e+02,5.000000e+02
NProcs,35041.0,1.200469e+03,5.968052e+02,1.186046e+02,7.697162e+02,1.166458e+03,1.583651e+03,6.073830e+03
ArrivedJobs,35041.0,9.978140e+00,3.150375e+00,0.000000e+00,8.000000e+00,1.000000e+01,1.200000e+01,2.500000e+01
CompletedJobs,35041.0,8.018378e+00,2.838550e+00,0.000000e+00,6.000000e+00,8.000000e+00,1.000000e+01,2.200000e+01
cosHour,35041.0,2.853800e-05,7.071270e-01,-1.000000e+00,-7.071068e-01,6.123234e-17,7.071068e-01,1.000000e+00
sinHour,35041.0,-1.163420e-17,7.071068e-01,-1.000000e+00,-7.071068e-01,0.000000e+00,7.071068e-01,1.000000e+00
cosDay,35041.0,2.843652e-05,7.071268e-01,-9.999630e-01,-7.086267e-01,4.303538e-03,7.025275e-01,1.000000e+00
sinDay,35041.0,1.178963e-05,7.071069e-01,-9.999907e-01,-7.055836e-01,6.432491e-16,7.055836e-01,9.999907e-01
